<a href="https://colab.research.google.com/github/fatusilva/test/blob/master/TFM_Modelo_BaseEncuestaCSAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TFM — CSAT (Notebook comentado)

Este cuaderno ha sido **anotado** para que el flujo quede claro para lectores y tutores.
- Secciones: *Importaciones, Montaje de Drive, Carga de datos, Limpieza, Ingeniería, Partición/Tuning, Entrenamiento, Evaluación, SHAP, Exportes*.
- Los comentarios `# === ... ===` **no alteran** la ejecución; solo documentan el propósito de cada bloque.
- Recomendación: ejecutar **Runtime → Restart & run all** para reproducibilidad.



# Trabajo Fin de Máster (TFM)
## Predicción de la satisfacción del cliente (CSAT) en un Contact Center
---


In [ ]:
# === Bloque misceláneo ===
!pip -q install xgboost imbalanced-learn
!wget https://github.com/fatusilva/test/raw/master/BaseEncuestasClientes.xlsx -O archivo.xlsx

In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
import numpy as np, json
import pandas as pd
import re
from datetime import datetime
#from google.colab import files

# Modelado / Métricas
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score,make_scorer, f1_score, recall_score,precision_score

# Opcional (plots)
import matplotlib.pyplot as plt
import seaborn as sns

# XGBoost
from xgboost import XGBClassifier
import xgboost as xgb

pd.set_option('display.max_colwidth', 120)
pd.set_option('display.max_columns', 200)

# ---- Helper: normalizar nombres de columnas y mapear a canónicos ----
import unicodedata

In [ ]:
# === Carga de datos: lee fuentes (CSV/Excel/Drive) al DataFrame ===
FILE_PATH = "https://github.com/fatusilva/test/raw/master/BaseEncuestasClientes.xlsx"
SHEET_NAME = None

# Ver qué hojas tiene el archivo
xl = pd.ExcelFile(FILE_PATH)
print("Hojas encontradas:", xl.sheet_names)

# Cargar la hoja (si SHEET_NAME es None usa la primera)
df = pd.read_excel(FILE_PATH, sheet_name=SHEET_NAME or 0, header=1)

# Normalizar nombres
df.columns = [str(c).strip() for c in df.columns]

print("Shape inicial:", df.shape)
print("Primeras columnas:", df.columns[:10].tolist())
df.head()



In [ ]:
# === Parsers robustos (reemplaza la celda de parsers anterior) ===
import re, numpy as np, pandas as pd, matplotlib.pyplot as plt

# Nombres que usás (confirmados):
CSAT_COL    = "CSAT General"
CANAL_COL   = "Origin_sf"
MOTIVO_COL   = "motivo_contacto"
PROD_COL    = "Producto_sf"
SUBPROD_COL = "Subproduct_sf"   # si no existe, lo detectamos y usamos PROD_COL
DUR_COL     = "Duración (en segundos)"

def _norm(s):
    # normaliza para matching tolerante (espacios invisibles, acentos, may/min)
    import unicodedata
    s = str(s)
    s = s.replace("\xa0"," ").strip()
    s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
    return s.lower()

# Mapa de nombres normalizados -> nombre real en df
norm_map = {_norm(c): c for c in df.columns}

def col(df, wanted):
    # busca una columna por nombre exacto, tolerando acentos/espacios/case
    if wanted in df.columns:
        return df[wanted], wanted
    key = _norm(wanted)
    if key in norm_map:
        real = norm_map[key]
        return df[real], real
    # fallback: intentar coincidencia por inicio (útil si hay sufijos raros)
    for k, real in norm_map.items():
        if k == key or k.startswith(key):
            return df[real], real
    raise KeyError(f"No encuentro la columna '{wanted}'. Revisa tildes/espacios. "
                   f"Ejemplos en df.columns: {list(df.columns)[:8]}")

# Construimos df_eda sin hacer selección por lista (evita KeyError por índices)
df_eda = pd.DataFrame(index=df.index)

# CSAT
s_csat, CSAT_REAL = col(df, CSAT_COL)
# Duración
try:
    s_dur, DUR_REAL = col(df, DUR_COL)
except KeyError:
    s_dur, DUR_REAL = pd.Series(np.nan, index=df.index), None
# Canal
s_canal, CANAL_REAL = col(df, CANAL_COL)
# Motivo
s_motivo, MOTIVO_REAL = col(df, MOTIVO_COL)
# Producto
s_prod, PROD_REAL = col(df, PROD_COL)
# Subproducto (si no está, usamos producto)
try:
    s_subp, SUBPROD_REAL = col(df, SUBPROD_COL)
except KeyError:
    s_subp, SUBPROD_REAL = s_prod, PROD_REAL

# Copiamos columnas “reales” (como están en tu df) y generamos auxiliares
df_eda[CSAT_REAL]   = s_csat
df_eda[CANAL_REAL]  = s_canal
df_eda[MOTIVO_REAL]  = s_motivo
df_eda[PROD_REAL]   = s_prod
df_eda[SUBPROD_REAL]= s_subp
if DUR_REAL is not None:
    df_eda[DUR_REAL] = s_dur

# --- Parsers ---
def parse_csat_to_1_5(s):
    s = s.astype(str).str.strip().str.lower()

    # Mapeo por texto (ajustá etiquetas si usás otras)
    mapa = {
        "muy insatisfecho": 1, "insatisfecho": 2, "neutral": 3,
        "satisfecho": 4, "muy satisfecho": 5,
        "muy malo": 1, "malo": 2, "regular": 3, "bueno": 4, "muy bueno": 5
    }
    out = s.map(mapa)

    # Para lo que no mapeó, intento extraer un número 1..5 (con punto o coma)
    mask_na = out.isna()
    if mask_na.any():
        s_na = s[mask_na].str.replace(",", ".", regex=False)
        # Captura SOLO el número (un único grupo) → devuelve una Serie, no un DF
        num = s_na.str.extract(r'([1-5](?:\.\d+)?)', expand=False)
        out.loc[mask_na] = pd.to_numeric(num, errors='coerce')

    # Redondeo y recorte al rango [1,5]
    out = pd.to_numeric(out, errors='coerce').round(0).clip(1, 5)
    return out


def parse_seconds(col):
    s = col.astype(str).str.strip()
    def to_sec_one(x):
        if not x or x.lower() in {"nan","none"}: return np.nan
        if ":" in x:
            x2 = x.replace(",", ".")
            parts = x2.split(":")
            try:
                parts = [float(p) for p in parts]
            except:
                return np.nan
            if len(parts) == 3:
                h,m,ss = parts; return h*3600 + m*60 + ss
            if len(parts) == 2:
                m,ss = parts;  return m*60 + ss
            return parts[0]
        try:
            return float(x.replace(",", "."))
        except:
            return np.nan
    return s.apply(to_sec_one)

# --- Auxiliares ---
df_eda["_csat_num"] = parse_csat_to_1_5(df_eda[CSAT_REAL])

if DUR_REAL is not None:
    df_eda["_dur_num"] = parse_seconds(df_eda[DUR_REAL])
else:
    df_eda["_dur_num"] = np.nan

# CSAT binario: 0 = <=3, 1 = >=4
df_eda["csat_bin"] = np.where(df_eda["_csat_num"] >= 4, 1, 0)

print("Columnas reales usadas ->",
      f"CSAT: {CSAT_REAL} | CANAL: {CANAL_REAL} | MOTIVO: {MOTIVO_REAL}| PROD: {PROD_REAL} | SUBPROD: {SUBPROD_REAL} | DUR: {DUR_REAL}")
print("Preview CSAT únicos:", sorted(pd.Series(df_eda["_csat_num"].dropna().unique()).tolist())[:10])

if DUR_REAL is not None:
    print("Duración (s) min/med/max:",
          df_eda["_dur_num"].min(skipna=True),
          df_eda["_dur_num"].median(skipna=True),
          df_eda["_dur_num"].max(skipna=True))
else:
    print("Duración no encontrada; Fig. 5 puede omitirse.")



## 2.3 Análisis Exploratorio de Datos (EDA) Visual

In [ ]:
# === EDA Visual: genera Fig3/4/5 y redacta "lecturas" sugeridas ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# Asume que ya existen: df_eda, CSAT_REAL, CANAL_REAL, PROD_REAL, SUBPROD_REAL, DUR_REAL
# y que df_eda tiene _csat_num, csat_bin, _dur_num (creados en la celda de parsers)

lecturas = []

# ---------------- FIGURA 3: CSAT por canal ----------------
df_canal = df_eda[[CANAL_REAL, "_csat_num", "csat_bin"]].dropna(subset=[CANAL_REAL, "_csat_num"]).copy()
msg3 = []

# Boxplot si hay ≥2 canales con ≥5 obs
groups, labels = [], []
for canal, sub in df_canal.groupby(CANAL_REAL):
    vals = sub["_csat_num"].dropna().values
    if len(vals) >= 5:
        groups.append(vals); labels.append(str(canal))

if len(groups) >= 2:
    fig = plt.figure(figsize=(9,5))
    plt.boxplot(groups, labels=labels, showfliers=False)
    plt.title("CSAT (1–5) por Canal")
    plt.xlabel("Canal"); plt.ylabel("CSAT (1–5)")
    plt.xticks(rotation=15, ha="right")
    plt.tight_layout(); plt.savefig("fig3_csat_por_canal.png", dpi=120); plt.close(fig)
    print("✅ fig3_csat_por_canal.png")
else:
    print("ℹ️ Boxplot omitido por bajo volumen por canal.")

# Barras: % insatisfechos (csat_bin=0) filtrando canales con ≥30 encuestas
grp = df_canal.groupby(CANAL_REAL)
counts = grp["csat_bin"].size()
mask = counts >= 30
rate = pd.Series(dtype=float)
if mask.any():
    rate = (grp["csat_bin"].apply(lambda s: (s==0).mean())[mask]).sort_values(ascending=False)
    fig = plt.figure(figsize=(9,5))
    rate.plot(kind="bar")
    plt.title("Tasa de insatisfacción por Canal (csat_bin=0) [≥30 encuestas]")
    plt.xlabel("Canal"); plt.ylabel("Porcentaje")
    plt.xticks(rotation=15, ha="right")
    plt.tight_layout(); plt.savefig("fig3b_tasa_insatisfaccion_por_canal.png", dpi=120); plt.close(fig)
    print("✅ fig3b_tasa_insatisfaccion_por_canal.png")
else:
    # Fallback: CSAT medio por canal (si no hay volumen para tasas)
    mean_by_c = grp["_csat_num"].mean().sort_values(ascending=True)
    if len(mean_by_c) > 0:
        fig = plt.figure(figsize=(9,5))
        mean_by_c.plot(kind="barh")
        plt.title("CSAT medio por Canal (fallback por bajo volumen)")
        plt.xlabel("CSAT (1–5)"); plt.ylabel("Canal")
        plt.tight_layout(); plt.savefig("fig3_fallback_csat_medio_por_canal.png", dpi=120); plt.close(fig)
        print("✅ fig3_fallback_csat_medio_por_canal.png")

# Lectura sugerida (se arma con mediana y con tasa si hubo)
medianas = grp["_csat_num"].median().sort_values()
if len(medianas) >= 1:
    worst_c = medianas.index[0]; worst_med = medianas.iloc[0]
    best_c  = medianas.index[-1]; best_med  = medianas.iloc[-1]
    msg3.append(f"Diferencias de distribución por canal: la mediana más baja de CSAT se observa en **{worst_c}** (≈ {worst_med:.2f}), mientras que la más alta en **{best_c}** (≈ {best_med:.2f}).")

if len(rate) > 0:
    top_r = rate.index[0]; top_rv = rate.iloc[0]; n_top = int(counts[top_r])
    msg3.append(f"En términos de tasa histórica de insatisfacción, **{top_r}** presenta el valor más alto (≈ {top_rv:.0%}, n={n_top}). Esto orienta acciones focalizadas por canal.")
else:
    msg3.append("Por bajo volumen en algunos canales, se reporta **CSAT medio por canal** como referencia.")

lecturas.append(("Figura 3 — CSAT por canal", " ".join(msg3)))

# ---------------- FIGURA 4: CSAT por producto/subproducto ----------------
cat_col = SUBPROD_REAL if SUBPROD_REAL in df_eda.columns else PROD_REAL
df_cat = df_eda[[cat_col, "csat_bin"]].dropna().copy()

min_n = 50  # umbral de volumen por categoría (ajustable)
counts_cat = df_cat[cat_col].value_counts()
keep = counts_cat[counts_cat >= min_n].index

msg4 = []
if len(keep) > 0:
    df_top = df_cat[df_cat[cat_col].isin(keep)].copy()
    rate_cat = df_top.groupby(cat_col)["csat_bin"].apply(lambda s: (s==0).mean()).sort_values(ascending=False)
    fig = plt.figure(figsize=(10,5))
    rate_cat.plot(kind="bar")
    plt.title(f"Tasa de insatisfacción por {cat_col} [≥{min_n} encuestas]")
    plt.xlabel(cat_col); plt.ylabel("Porcentaje (csat_bin=0)")
    plt.xticks(rotation=30, ha="right")
    plt.tight_layout(); plt.savefig("fig4_csat_por_producto_subproducto.png", dpi=120); plt.close(fig)
    print("✅ fig4_csat_por_producto_subproducto.png")

    # Top 3 para redactar
    topN = rate_cat.head(3)
    toplist = [f"{idx} ({val:.0%})" for idx, val in topN.items()]
    msg4.append("Se evidencian categorías con **tasa histórica** mayor de insatisfacción: " + ", ".join(toplist) + ".")
    msg4.append("Esto justifica el uso de **target mean/frequency encoding** a nivel de producto/subproducto, que luego emerge como relevante en SHAP.")
else:
    msg4.append(f"Omitido por volumen bajo: ninguna categoría con ≥{min_n} encuestas. Si necesitás graficar, bajá el umbral a 30.")

lecturas.append(("Figura 4 — CSAT por producto/subproducto", " ".join(msg4)))

# ---------------- FIGURA 5: Duración (s) vs CSAT ----------------
msg5 = []
if DUR_REAL is not None:
    dur = df_eda[["_dur_num", "_csat_num", "csat_bin"]].dropna().copy()
    if len(dur) >= 50:
        # Recorte de outliers (p99.5)
        p995 = dur["_dur_num"].quantile(0.995)
        dur = dur[dur["_dur_num"] <= p995]

        dur["dur_bin"] = pd.qcut(dur["_dur_num"], q=10, duplicates="drop")
        summary = dur.groupby("dur_bin").agg(
            csat_mean=("_csat_num", "mean"),
            insat_rate=("csat_bin", lambda s: (s==0).mean()),
            dur_med=("_dur_num", "median")
        ).reset_index().sort_values("dur_med")

        # plot
        fig = plt.figure(figsize=(9,5))
        ax = plt.gca()
        ax.plot(summary["dur_med"], summary["csat_mean"], marker="o", label="CSAT medio (1–5)")
        ax.set_xlabel("Duración (s) - mediana por decil"); ax.set_ylabel("CSAT medio (1–5)")
        ax2 = ax.twinx()
        ax2.plot(summary["dur_med"], summary["insat_rate"], marker="s", linestyle="--", label="Tasa insatisfacción")
        ax2.set_ylabel("Tasa insatisfacción (0–1)")
        plt.title("Relación Duración (s) – CSAT / Tasa de Insatisfacción")
        lines, labels_ = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels_ + labels2, loc="best")
        plt.tight_layout(); plt.savefig("fig5_duracion_vs_csat.png", dpi=120); plt.close(fig)
        print("✅ fig5_duracion_vs_csat.png")

        # lectura cuantitativa
        first = summary.iloc[0]; last = summary.iloc[-1]
        delta = last["csat_mean"] - first["csat_mean"]
        msg5.append(f"**Duraciones muy cortas** (≈ {first['dur_med']:.0f}s) muestran CSAT medio ≈ {first['csat_mean']:.2f};")
        msg5.append(f"a medida que la duración crece (≈ {last['dur_med']:.0f}s), el CSAT medio sube a ≈ {last['csat_mean']:.2f} (Δ ≈ {delta:.2f}).")
        msg5.append("La hipótesis es que mayor tiempo efectivo permite resolver mejor, consistente con la importancia de **Duración (s)** observada en SHAP.")
    else:
        msg5.append("Omitido: no hay suficientes datos no nulos para duración (≥50).")
else:
    msg5.append("No se encontró la columna de Duración; esta figura puede omitirse.")

lecturas.append(("Figura 5 — Duración (s) vs CSAT", " ".join(msg5)))

# ---------------- Mostrar "lecturas" listas para pegar ----------------
print("\n\n==== LECTURAS SUGERIDAS (copiar/pegar bajo cada figura en el TFM) ====\n")
for titulo, texto in lecturas:
    print(f"{titulo}\n{texto}\n".replace("{text}", texto))



#### Explicabilidad con SHAP

In [ ]:
# === Explicabilidad con SHAP ===
# Importancia global (bar/beeswarm) y dependence plots para variables top.
# Detectar y convertir columnas de fechas si existen
posibles_fecha_encuesta = ['Fecha registrada', 'Fecha Registrada']
fecha_encuesta_col = next((c for c in posibles_fecha_encuesta if c in df.columns), None)

if fecha_encuesta_col:
    df[fecha_encuesta_col] = pd.to_datetime(df[fecha_encuesta_col], errors='coerce')

# cuenta_creada (alta del cliente)
if 'cuenta_creada' in df.columns:
    df['cuenta_creada'] = pd.to_datetime(df['cuenta_creada'], errors='coerce')

# Antigüedad del cliente en meses (aprox; días/30.44)
if fecha_encuesta_col and 'cuenta_creada' in df.columns:
    diff_days = (df[fecha_encuesta_col] - df['cuenta_creada']).dt.days
    df['antiguedad_cliente_meses'] = (diff_days / 30.44).clip(lower=0)
else:
    df['antiguedad_cliente_meses'] = np.nan

# Crear rangos de antigüedad
bins = [0, 6, 12, 24, 60, 120, float('inf')]
labels = ['<6m', '6-12m', '1-2a', '2-5a', '5-10a', '10+a']
df['antiguedad_rango'] = pd.cut(df['antiguedad_cliente_meses'], bins=bins, labels=labels)

# Derivados temporales (si hay fecha de encuesta)
if fecha_encuesta_col:
    df['mes_encuesta'] = df[fecha_encuesta_col].dt.month
    df['dia_semana_encuesta'] = df[fecha_encuesta_col].dt.weekday
    df['hora_encuesta'] = df[fecha_encuesta_col].dt.hour

print("Shape inicial:", df.shape)
print("Primeras columnas:", df.columns[:50].tolist())
df.head()






# Exploración visual
*   ### CSAT promedio por rango de antigüedad:

In [ ]:
# === Bloque misceláneo ===
# Asegurarse de que CSAT sea numérico
df['CSAT General'] = pd.to_numeric(df['CSAT General'], errors='coerce')


In [ ]:
# === Visualización (EDA/Resultados) ===
# Gráficos para entender distribución, relaciones y resultados del modelo.
#Visualizar la distribucion del CSAT por diferentes variable

# Promedio de CSAT por rango
csat_por_antig = df.groupby('antiguedad_rango')['CSAT General'].mean()
csat_por_producto = df.groupby('grupo_producto')['CSAT General'].mean()
csat_por_producto = df.groupby('motivo_contacto')['CSAT General'].mean()

# Gráfico
csat_por_antig.plot(kind='bar', title='Promedio de CSAT por antigüedad del cliente', color='skyblue')
plt.ylabel('CSAT promedio')
plt.xlabel('Rango de antigüedad')
plt.grid(True)
plt.tight_layout()
plt.show()






*   ## Analisis de nulos




In [ ]:
# === Explicabilidad con SHAP ===
# Importancia global (bar/beeswarm) y dependence plots para variables top.
nulos = df.isnull().sum().to_frame(name='Cantidad de nulos')
nulos['% de nulos'] = (nulos['Cantidad de nulos'] / len(df) * 100).round(2)
nulos = nulos[nulos['Cantidad de nulos'] > 0].sort_values(by='% de nulos', ascending=False)
display(nulos.head(100))
df.isnull().sum()*100/df.shape[0]


## Limpieza / Normalización: manejo de nulos, tipos, merges y depuración
##### Objetivo: dejar los datos consistentes antes del EDA/modelado.

In [ ]:
# CSAT General -> binaria: 1 si CSAT >= 4, 0 en caso contrario
assert 'CSAT General' in df.columns, "No se encontró 'CSAT General' en el dataset."
df['CSAT_bin'] = (df['CSAT General'] >= 4).astype(int)

#Eliminamos variables de fuga relacionadas a resolución (si están)
fuga_patterns = ['resoluc', 'Respuesta Resolucion','¿Resolvimos tu consulta?']
cols_fuga = [c for c in df.columns if any(pat.lower() in c.lower() for pat in fuga_patterns)]
df = df.drop(columns=cols_fuga, errors='ignore')

print("Columnas de fuga eliminadas:", cols_fuga)
print("Distribución CSAT_bin:\n", df['CSAT_bin'].value_counts(normalize=True).round(4)*100)


In [ ]:
# Elegir columna origen
posibles_motivo = ['motivo_contacto']
motivo_col = next((c for c in posibles_motivo if c in df.columns), None)

#Agrupamos los motivos de contacto
def agrupar_motivos_texto(valor):
    if pd.isna(valor):
        return 'Otro'
    v = str(valor).lower()
    # Tarjeta
    if any(k in v for k in ['tarjeta', 'plástico', 'pin', 'visa', 'mastercard', 'extravio', 'bloqueo']):
        return 'Tarjeta'
    # Préstamos / créditos
    if any(k in v for k in ['préstam', 'prestam', 'cuota', 'sobregiro', 'credito', 'crédito']):
        return 'Préstamos'
    # Transferencias / pagos / QR / billeteras / ATM
    if any(k in v for k in ['transferen', 'pago', 'qr', 'ted', 'atm', 'alias', 'debito', 'débito', 'swift', 'cnb']):
        return 'Transferencias/ Pagos'
    # Cuenta / App / Acceso / Onboarding
    if any(k in v for k in ['cuenta', 'onboarding', 'contraseña', 'password', 'app', 'login', 'web', 'canal']):
        return 'Cuenta/App'
    # Ofertas / Comercial / Promos
    if any(k in v for k in ['oferta', 'promoc', 'comercial', 'venta', 'promo','loyal']):
        return 'Ofertas/Comercial'
    # Fraude / seguridad
    if any(k in v for k in ['fraude', 'estafa', 'seguridad', 'vulneración', 'robo']):
        return 'Fraude/Seguridad'
    # Inversiones / CDA / fondos / casa de bolsa
    if any(k in v for k in ['cda', 'fondo', 'invers', 'bolsa']):
        return 'Inversiones'
    # Servicios / facturas
    if any(k in v for k in ['servicio', 'factura', 'facturador']):
        return 'Servicios'
    # Generales / Operativos
    if any(k in v for k in ['consulta', 'requisito', 'condicion', 'soporte', 'ayuda', 'informacion', 'información']):
        return 'Operativo/Soporte'
    return 'Otro'

if motivo_col:
    if df[motivo_col].dtype == object:
        df['motivo_contacto_categoria'] = df[motivo_col].apply(agrupar_motivos_texto)
    else:
        # Si no tenés el texto original, vas a agrupar a 'Otro'
        df['motivo_contacto_categoria'] = 'Otro'
else:
    df['motivo_contacto_categoria'] = 'Otro'

df['motivo_contacto_categoria'] = df['motivo_contacto_categoria'].fillna('Otro')

print("Top categorías motivo_contacto_categoria:")
print(df['motivo_contacto_categoria'].value_counts().head(10))


In [ ]:
# === Bloque misceláneo ===
for name in ['X','y','X_train','X_test','y_train','y_test','X_train_scaled','X_test_scaled']:
    if name in globals():
        del globals()[name]

##Ingeniería de variables: codificaciones y derivados (freq/target mean, one-hot, etc.)
#### Nota: evita data leakage (calcula sobre training o usa pipelines/column_transformers).

In [ ]:
# 1) Lista blanca de columnas (ajustá nombres según existan en tu df)
whitelist = [
    'antiguedad_cliente_meses','rango_etario','genero','ya_es_cliente','Recontacto_sf','Producto_sf','Subproduct_sf','motivo_contacto_categoria'
]

# 2) Construir df_modelo solo con whitelist + target
cols_existentes = [c for c in whitelist if c in df.columns]
df_modelo = df[cols_existentes + ['CSAT_bin']].copy()

print("✅ Columnas usadas:", df_modelo.columns.tolist())

# 3) (doble seguro) No permitir CSAT* ni Resolucion* en features
ban_patterns = ['csat', 'satisf', 'resoluc']
cols_ban = [c for c in df_modelo.columns if any(p in c.lower() for p in ban_patterns) and c != 'CSAT_bin']
if cols_ban:
    df_modelo = df_modelo.drop(columns=cols_ban)
    print("🚫 Quitadas por seguridad:", cols_ban)

In [ ]:
# 4) Separar X/y
X = df_modelo.drop(columns=['CSAT_bin']).copy()
y = df_modelo['CSAT_bin'].astype(int)

# 5) Codificar solo las categóricas que queden en X
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
print("🔤 Categóricas codificadas:", cat_cols)

# 6) Imputación (por si queda algún NaN)
imp = SimpleImputer(strategy='mean')
X = pd.DataFrame(imp.fit_transform(X), columns=X.columns)

# 7) Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print("✅ X_train:", X_train.shape, "y_train:", y_train.shape)
print("✅ X_test:", X_test.shape, "y_test:", y_test.shape)

print("📊 y_train %:\n", (y_train.value_counts(normalize=True)*100).round(2))


resumen = pd.DataFrame({
    'Variable': X.columns,
    'dtype': X.dtypes.values,
    '% nulos': (X.isnull().mean()*100).round(2),
    'n_únicos': [X[c].nunique() for c in X.columns]
}).sort_values('% nulos', ascending=False)
resumen.head(20)

#### Explicabilidad con SHAP

In [ ]:
# Importancia global (bar/beeswarm) y dependence plots para variables top.
print("✅ Columnas usadas:", df_modelo.columns.tolist())

# Análisis de valores nulos
nulos = df_modelo.isnull().sum().to_frame(name='Cantidad de nulos')
nulos['% de nulos'] = (nulos['Cantidad de nulos'] / len(df) * 100).round(2)
nulos = nulos[nulos['Cantidad de nulos'] > 0].sort_values(by='% de nulos', ascending=False)
nulos.head(20)

print("🔧 Features usadas por el modelo:", X.columns.tolist())
print("Shape X:", X.shape, " | Shape y:", y.shape)
print("\nTipos de datos:")
print(X.dtypes.value_counts())

In [ ]:
# Eliminar columnas con 100% NaNs
columnas_100_nan = X_train.columns[X_train.isnull().all()]
print("🧼 Columnas eliminadas por tener 100% NaNs:", columnas_100_nan.tolist())

X_train = X_train.drop(columns=columnas_100_nan)
X_test = X_test.drop(columns=columnas_100_nan, errors='ignore')

#**Entrenamiento del modelo**

###Regresión logística (simple y fácil de interpretar)

In [ ]:
# Define el estimador (e.g., XGBoost/RandomForest/Logística) y entrena con los features preparados.
#Paso: Entrenamiento y evaluación de modelo base
#Usaremos:
#Regresión logística (simple y fácil de interpretar)
#Métricas: accuracy, precision, recall, f1_score, roc_auc
#Matriz de confusión

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

modelo_log = LogisticRegression(max_iter=8000)
modelo_log.fit(X_train_scaled, y_train)

y_pred_log = modelo_log.predict(X_test_scaled)
y_proba_log = modelo_log.predict_proba(X_test_scaled)[:, 1]

print("📌 Clasification Report — Regresión Logística (sin resolución):")
print(classification_report(y_test, y_pred_log))
print("📈 ROC-AUC:", roc_auc_score(y_test, y_proba_log))
print("--")
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_log)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['No Satisfactorio', 'Satisfactorio'], yticklabels=['No Satisfactorio', 'Satisfactorio'])
plt.title("Matriz de Confusión — Logística (sin resolución)")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()


In [ ]:
# === Visualización (EDA/Resultados) ===
# Gráficos para entender distribución, relaciones y resultados del modelo.
# Gráfico para visualizar la relación entre Recontacto y CSAT
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='CSAT General', hue='Recontacto_sf')
plt.title('Distribución de CSAT General según si hubo Recontacto')
plt.xlabel('Puntuación CSAT General')
plt.ylabel('Cantidad de Clientes')
plt.show()

In [ ]:
# === Visualización (EDA/Resultados) ===
# Gráficos para entender distribución, relaciones y resultados del modelo.
#Extraer el coeficiente
coeficientes = modelo_log.coef_[0]
variables = X_train.columns

#Crear dataframe
df_coef = pd.DataFrame({
    'Variable': variables,
    'Coeficiente': coeficientes
}).sort_values(by='Coeficiente', ascending=False)

# 🔝 Mostrar top 15 positivos y negativos
top_pos = df_coef.head(15)
top_neg = df_coef.tail(15)

print(top_pos)

# 📊 Gráfico de barras
plt.figure(figsize=(10, 8))
plt.barh(top_pos['Variable'], top_pos['Coeficiente'], color='green')
plt.title("🔼 Variables que aumentan la probabilidad de satisfacción (CSAT=1)")
plt.xlabel("Coeficiente")
plt.tight_layout()
plt.show()


print(top_pos)

plt.figure(figsize=(10, 8))
plt.barh(top_neg['Variable'], top_neg['Coeficiente'], color='red')
plt.title("🔽 Variables que disminuyen la probabilidad de satisfacción (CSAT=1)")
plt.xlabel("Coeficiente")
plt.tight_layout()
plt.show()

####RandomForest

In [ ]:
# === Entrenamiento del modelo ===
# Define el estimador (e.g., XGBoost/RandomForest/Logística) y entrena con los features preparados.
modelo_rf = RandomForestClassifier(n_estimators=200, random_state=42)
modelo_rf.fit(X_train, y_train)

y_pred_rf = modelo_rf.predict(X_test)
y_proba_rf = modelo_rf.predict_proba(X_test)[:, 1]

print("📌 Clasification Report — Random Forest (sin resolución):")
print(classification_report(y_test, y_pred_rf))
print("📈 ROC-AUC:", roc_auc_score(y_test, y_proba_rf))
print("----")

cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['No Satisfactorio', 'Satisfactortio'], yticklabels=['No Satisfactorio', 'Satisfactorio'])
plt.title("Matriz de Confusión — RF (sin resolución)")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()


**XGBoost con pesos + early stopping**

In [ ]:
# ===== XGBoost con early stopping (API nativa) + pesos clase 0 + ajuste de umbral =====
print("XGBoost version:", xgb.__version__)

# 1) Split interno para validación
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.10, random_state=42, stratify=y_train
)

# 2) Pesos para favorecer la clase 0 (insatisfechos)
ratio = (y_tr == 1).sum() / max(1, (y_tr == 0).sum())
sample_w = np.where(y_tr == 0, ratio, 1.0)

# 3) DMatrix (requerido por API nativa)
dtrain = xgb.DMatrix(X_tr, label=y_tr, weight=sample_w)
dvalid = xgb.DMatrix(X_val, label=y_val)
dtest  = xgb.DMatrix(X_test)

# 4) Parámetros equivalentes
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 5,
    'eta': 0.05,                # = learning_rate
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'min_child_weight': 1,
    'lambda': 1.0,              # = reg_lambda
    'alpha': 0.0,               # = reg_alpha
    'tree_method': 'hist',
    'seed': 42
}

# 5) Entrenamiento con early stopping estable
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
booster = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=2000,
    evals=watchlist,
    early_stopping_rounds=50,
    verbose_eval=False
)
print(f"Mejor iteración: {booster.best_iteration} | Mejor logloss(valid): {booster.best_score:.4f}")

# 6) Predicción en TEST respetando mejor iteración
try:
    y_proba = booster.predict(dtest, iteration_range=(0, booster.best_iteration + 1))
except TypeError:
    # Fallback por compatibilidad
    y_proba = booster.predict(dtest)

# --- Report con umbral 0.50 ---
y_pred = (y_proba >= 0.50).astype(int)
print("📌 Report (umbral 0.50):")
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['No Sat', 'Sat'], yticklabels=['No Sat', 'Sat'])
plt.title("Matriz — XGB (umbral 0.50)")
plt.xlabel("Predicción"); plt.ylabel("Real"); plt.show()

# 7) Búsqueda de umbral óptimo para maximizar F1 de la clase 0
def umbral_optimo_clase0(y_true, proba_pos):
    best = {'thr':0.5, 'f1_0':-1, 'rec0':0, 'prec0':0}
    for thr in np.linspace(0.1, 0.9, 81):
        y_hat = (proba_pos >= thr).astype(int)        # proba de clase 1
        f1_0  = f1_score(y_true, y_hat, pos_label=0)
        rec0  = recall_score(y_true, y_hat, pos_label=0)
        prec0 = precision_score(y_true, y_hat, pos_label=0)
        if f1_0 > best['f1_0']:
            best = {'thr':thr, 'f1_0':f1_0, 'rec0':rec0, 'prec0':prec0}
    return best

best = umbral_optimo_clase0(y_test, y_proba)
print("🔎 Umbral óptimo para clase 0:", best)



#### Limpieza / Normalización: manejo de nulos, tipos, merges y depuración

In [ ]:
# Objetivo: dejar los datos consistentes antes del EDA/modelado.
# --- Report con umbral óptimo para clase 0 ---
y_pred_opt = (y_proba >= best['thr']).astype(int)
print("📌 Report (umbral óptimo clase 0):")
print(classification_report(y_test, y_pred_opt))

cm_opt = confusion_matrix(y_test, y_pred_opt)
sns.heatmap(cm_opt, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['No Sat', 'Sat'], yticklabels=['No Sat', 'Sat'])
plt.title("Matriz — XGB (umbral óptimo clase 0)")
plt.xlabel("Predicción"); plt.ylabel("Real"); plt.show()

In [ ]:
# elegir umbral por objetivo de negocio o costo

def umbral_por_objetivo(y_true, proba_pos, min_recall0=0.70):
    mejor = {'thr':0.5, 'rec0':0, 'f1_0':-1}
    for thr in np.linspace(0.1, 0.9, 81):
        y_hat = (proba_pos >= thr).astype(int)
        rec0  = recall_score(y_true, y_hat, pos_label=0)
        f1_0v = f1_score(y_true, y_hat, pos_label=0)
        if rec0 >= min_recall0 and f1_0v > mejor['f1_0']:
            mejor = {'thr':thr, 'rec0':rec0, 'f1_0':f1_0v}
    return mejor

def umbral_por_coste(y_true, proba_pos, cost_fn=5.0, cost_fp=1.0):
    mejor = {'thr':0.5, 'costo':float('inf')}
    for thr in np.linspace(0.1, 0.9, 81):
        y_hat = (proba_pos >= thr).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
        costo = cost_fn*fn + cost_fp*fp
        if costo < mejor['costo']:
            mejor = {'thr':thr, 'costo':costo}
    return mejor

# Usa y_proba de la celda XGB nativa
tgt = umbral_por_objetivo(y_test, y_proba, min_recall0=0.70)
print("🎯 Umbral con recall0 ≥ 0.70:", tgt)

y_hat_obj = (y_proba >= tgt['thr']).astype(int)
print(classification_report(y_test, y_hat_obj))


In [ ]:
# XGBoost RandomizedSearchCV pro-clase 0 (opcional)

# Pesos para favorecer clase 0 en el fit
w_ratio = (y_train==1).sum() / (y_train==0).sum()
sw = np.where(y_train==0, w_ratio, 1.0)

# Scorers que priorizan la clase 0
f1_0     = make_scorer(f1_score,    pos_label=0)
recall_0 = make_scorer(recall_score,pos_label=0)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

xgb_base = XGBClassifier(
    eval_metric='logloss',
    tree_method='hist',
    n_jobs=-1,
    random_state=42
)

param_distributions = {
    'n_estimators':     [300, 600, 900, 1200],
    'learning_rate':    [0.03, 0.05, 0.08, 0.1],
    'max_depth':        [4, 5, 6, 8],
    'min_child_weight': [1, 2, 5],
    'subsample':        [0.7, 0.85, 1.0],
    'colsample_bytree': [0.7, 0.85, 1.0],
    'gamma':            [0, 0.3, 1.0],
    'reg_lambda':       [1.0, 3.0, 6.0],
    'reg_alpha':        [0.0, 0.1, 0.5]
}

rand = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_distributions,
    n_iter=30,
    scoring={'f1_0': f1_0, 'recall_0': recall_0, 'bacc': 'balanced_accuracy'},
    refit='f1_0',
    cv=cv,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

rand.fit(X_train, y_train, sample_weight=sw)
print("🔧 Mejores hiperparámetros:", rand.best_params_)
best_xgb = rand.best_estimator_

# Evaluación en test con umbral 0.50
y_proba_rs = best_xgb.predict_proba(X_test)[:,1]
y_pred_rs  = (y_proba_rs >= 0.50).astype(int)
print("📌 Report Randomized (0.50):")
print(classification_report(y_test, y_pred_rs))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_rs))

# (opcional) aplicar el mismo criterio de umbral que en la celda A:
tgt_rs = umbral_por_objetivo(y_test, y_proba_rs, min_recall0=0.70)
print("🎯 Umbral RS con recall0 ≥ 0.70:", tgt_rs)
y_hat_rs = (y_proba_rs >= tgt_rs['thr']).astype(int)
print(classification_report(y_test, y_hat_rs))


In [ ]:
# === Limpieza / Normalización: manejo de nulos, tipos, merges y depuración ===
# Objetivo: dejar los datos consistentes antes del EDA/modelado.
# Elegí en cuáles columnas aplicar encoding adicional (de las que TENÉS en X)
cand_cols = [c for c in ['Producto_sf','Subproduct_sf','grupo_producto',
                         'motivo_contacto_categoria','rango_etario','genero',
                         'ya_es_cliente']
             if c in X_train.columns]

def add_freq_and_target_mean(Xtr, Xte, ytr, cols, positive_class=0):
    Xtr = Xtr.copy(); Xte = Xte.copy()
    for col in cols:
        # Frequency encoding (proporción de cada categoría en train)
        freq = Xtr[col].value_counts(normalize=True)
        Xtr[f'{col}_freq'] = Xtr[col].map(freq)
        Xte[f'{col}_freq'] = Xte[col].map(freq).fillna(0)

        # Target-mean encoding para clase 0 (prob de ser clase 0 por categoría)
        # Nota: calculado SOLO en train para evitar fuga
        tmp = pd.DataFrame({col: Xtr[col], 'y': ytr})
        # prob de clase 0:
        mean0 = tmp.groupby(col)['y'].apply(lambda v: (v==positive_class).mean())
        global_mean0 = (ytr==positive_class).mean()
        Xtr[f'{col}_mean0'] = Xtr[col].map(mean0)
        Xte[f'{col}_mean0'] = Xte[col].map(mean0).fillna(global_mean0)

    return Xtr, Xte

X_train_ext, X_test_ext = add_freq_and_target_mean(X_train, X_test, y_train, cand_cols, positive_class=0)
print("Columns added:",
      [c for c in X_train_ext.columns if c.endswith('_freq') or c.endswith('_mean0')])
print("Shapes -> X_train_ext:", X_train_ext.shape, "| X_test_ext:", X_test_ext.shape)


In [ ]:
# === Bloque misceláneo ===
print("X_train cols:", X_train.columns.tolist())

candidatas = ['Producto_sf','Subproduct_sf','grupo_producto',
              'motivo_contacto_categoria','rango_etario','genero','ya_es_cliente',
              'ciudad','Canal','canal_ticket']
faltan = [c for c in candidatas if c not in X_train.columns]
print("❗ No están en X_train:", faltan)

In [ ]:
# === Limpieza / Normalización: manejo de nulos, tipos, merges y depuración ===
# Objetivo: dejar los datos consistentes antes del EDA/modelado.

# Split interno para early stopping
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_ext, y_train, test_size=0.10, random_state=42, stratify=y_train
)

# Pesos a clase 0 (minoritaria)
ratio = (y_tr==1).sum() / max(1, (y_tr==0).sum())
sw = np.where(y_tr==0, ratio, 1.0)

# DMatrix
dtrain = xgb.DMatrix(X_tr, label=y_tr, weight=sw)
dvalid = xgb.DMatrix(X_val, label=y_val)
dtest  = xgb.DMatrix(X_test_ext)

# Parámetros (conservadores, buenos para recall_0)
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 5,
    'eta': 0.05,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'min_child_weight': 2,
    'lambda': 1.0,
    'alpha': 0.5,
    'tree_method': 'hist',
    'seed': 42
}

booster = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=2000,
    evals=[(dtrain,'train'), (dvalid,'valid')],
    early_stopping_rounds=50,
    verbose_eval=False
)

print(f"Mejor iteración: {booster.best_iteration} | Mejor logloss(valid): {booster.best_score:.4f}")

# Predicciones test
y_proba_ext = booster.predict(dtest, iteration_range=(0, booster.best_iteration + 1))
y_pred05    = (y_proba_ext >= 0.50).astype(int)

print("📌 Report (umbral 0.50) con features extra:")
print(classification_report(y_test, y_pred05))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_ext))

cm = confusion_matrix(y_test, y_pred05)
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['No Sat','Sat'], yticklabels=['No Sat','Sat'])
plt.title("Matriz — XGB + (freq + mean0) — umbral 0.50")
plt.xlabel("Predicción"); plt.ylabel("Real"); plt.show()


In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
import numpy as np
from sklearn.metrics import classification_report, recall_score, f1_score, confusion_matrix

def umbral_por_objetivo(y_true, proba_pos, min_recall0=0.70):
    mejor = {'thr':0.5, 'rec0':0, 'f1_0':-1}
    for thr in np.linspace(0.1, 0.9, 81):
        y_hat = (proba_pos >= thr).astype(int)
        rec0  = recall_score(y_true, y_hat, pos_label=0)
        f1_0v = f1_score(y_true, y_hat, pos_label=0)
        if rec0 >= min_recall0 and f1_0v > mejor['f1_0']:
            mejor = {'thr':thr, 'rec0':rec0, 'f1_0':f1_0v}
    return mejor

best_obj = umbral_por_objetivo(y_test, y_proba_ext, min_recall0=0.70)
print("🎯 Umbral con recall0 ≥ 0.70:", best_obj)

y_hat_obj = (y_proba_ext >= best_obj['thr']).astype(int)
print("📌 Report (umbral objetivo):")
print(classification_report(y_test, y_hat_obj))

cm2 = confusion_matrix(y_test, y_hat_obj)
sns.heatmap(cm2, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['No Sat','Sat'], yticklabels=['No Sat','Sat'])
plt.title("Matriz — XGB + (freq + mean0) — umbral objetivo")
plt.xlabel("Predicción"); plt.ylabel("Real"); plt.show()


In [ ]:
# === Bloque misceláneo ===
#Para evitar que variables viejas se mezclen, podés limpiar algunas antes de empezar la parte nueva:

for name in ['X_train_ext','X_test_ext','y_proba','y_pred']:
    if name in globals(): del globals()[name]


In [ ]:
# === Limpieza / Normalización: manejo de nulos, tipos, merges y depuración ===
# Objetivo: dejar los datos consistentes antes del EDA/modelado.
# Celda NUEVA: features extra
cand_cols = [c for c in [
    'Producto_sf','Subproduct_sf','grupo_producto','motivo_contacto_categoria',
    'rango_etario','genero','ya_es_cliente','ciudad','Canal','canal_ticket'
] if c in X_train.columns]

def add_freq_and_target_mean(Xtr, Xte, ytr, cols, positive_class=0):
    Xtr = Xtr.copy(); Xte = Xte.copy()
    base_p0 = (ytr == positive_class).mean()  # prob global de clase 0

    for col in cols:
        # Frequency encoding
        freq = Xtr[col].value_counts(normalize=True)
        Xtr[f'{col}_freq'] = Xtr[col].map(freq)
        Xte[f'{col}_freq'] = Xte[col].map(freq).fillna(0)

        # Target-mean encoding (prob de ser clase 0)
        tmp = pd.DataFrame({col: Xtr[col], 'y': ytr.values})
        # y es 0/1 → prob0 = 1 - mean(y)
        mean1 = tmp.groupby(col)['y'].mean()        # P(y=1 | col)
        prob0 = 1.0 - mean1                         # P(y=0 | col)
        Xtr[f'{col}_mean0'] = Xtr[col].map(prob0)
        Xte[f'{col}_mean0'] = Xte[col].map(prob0).fillna(base_p0)

    return Xtr, Xte


X_train_ext, X_test_ext = add_freq_and_target_mean(X_train, X_test, y_train, cand_cols, positive_class=0)

print("➕ Nuevas columnas:", [c for c in X_train_ext.columns if c.endswith('_freq') or c.endswith('_mean0')])
print("📐 Shapes:", X_train_ext.shape, X_test_ext.shape)


In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
# Celda NUEVA: XGBoost nativo + early stopping con X_train_ext/X_test_ext
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, f1_score, recall_score
import numpy as np, seaborn as sns, matplotlib.pyplot as plt

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_ext, y_train, test_size=0.10, random_state=42, stratify=y_train
)
ratio = (y_tr==1).sum() / max(1,(y_tr==0).sum())
sw = np.where(y_tr==0, ratio, 1.0)

dtrain = xgb.DMatrix(X_tr,  label=y_tr, weight=sw)
dvalid = xgb.DMatrix(X_val, label=y_val)
dtest  = xgb.DMatrix(X_test_ext)

params = {
    'objective':'binary:logistic','eval_metric':'logloss','max_depth':5,'eta':0.05,
    'subsample':0.9,'colsample_bytree':0.9,'min_child_weight':2,'lambda':1.0,'alpha':0.5,
    'tree_method':'hist','seed':42
}
booster = xgb.train(params, dtrain, num_boost_round=2000,
                    evals=[(dtrain,'train'),(dvalid,'valid')],
                    early_stopping_rounds=50, verbose_eval=False)

print(f"Mejor iteración: {booster.best_iteration} | logloss(valid): {booster.best_score:.4f}")

y_proba = booster.predict(dtest, iteration_range=(0, booster.best_iteration+1))
y_pred  = (y_proba >= 0.50).astype(int)
print("📌 Report (0.50):"); print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Oranges',
            xticklabels=['No Sat','Sat'], yticklabels=['No Sat','Sat'])
plt.title("Matriz — XGB + encodings (umbral 0.50)"); plt.show()

# Umbral por objetivo de negocio (ej. recall0 ≥ 0.70)
def umbral_por_objetivo(y_true, proba_pos, min_recall0=0.70):
    best = {'thr':0.5, 'rec0':0, 'f1_0':-1}
    for thr in np.linspace(0.1,0.9,81):
        y_hat = (proba_pos >= thr).astype(int)
        rec0  = recall_score(y_true, y_hat, pos_label=0)
        f1_0v = f1_score(y_true, y_hat, pos_label=0)
        if rec0 >= min_recall0 and f1_0v > best['f1_0']:
            best = {'thr':thr, 'rec0':rec0, 'f1_0':f1_0v}
    return best

best_obj = umbral_por_objetivo(y_test, y_proba, min_recall0=0.70)
print("🎯 Umbral con recall0 ≥ 0.70:", best_obj)
y_hat_obj = (y_proba >= best_obj['thr']).astype(int)
print("📌 Report (umbral objetivo):"); print(classification_report(y_test, y_hat_obj))


In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

umbral_final = 0.58  # ajustalo con el valor que te salió (p.ej. 0.58)
y_pred_final = (y_proba >= umbral_final).astype(int)

print(f"👉 Umbral fijado: {umbral_final:.2f}")
print(classification_report(y_test, y_pred_final))

tn, fp, fn, tp = confusion_matrix(y_test, y_pred_final).ravel()
print("Confusión:", {'tn':int(tn),'fp':int(fp),'fn':int(fn),'tp':int(tp)})
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

resultados = {
    'umbral': umbral_final,
    'tn': int(tn), 'fp': int(fp), 'fn': int(fn), 'tp': int(tp),
    'auc': float(roc_auc_score(y_test, y_proba))
}
resultados


In [ ]:
# === Bloque misceláneo ===
# Importancias (ganancia) del booster nativo
imp = booster.get_score(importance_type='gain')
top10 = sorted(imp.items(), key=lambda x: -x[1])[:10]
print("🔝 Top 10 features (gain):")
for k,v in top10:
    print(f"{k}: {v:.4f}")


In [ ]:
# === Limpieza / Normalización: manejo de nulos, tipos, merges y depuración ===
# Objetivo: dejar los datos consistentes antes del EDA/modelado.
# 🔒 Experimento A/B: quitar Recontacto_sf y reentrenar rápido sobre las features extendidas
cols_sin_recontacto = [c for c in X_train_ext.columns if c != 'Recontacto_sf']
dtrain_ab = xgb.DMatrix(X_tr[cols_sin_recontacto], label=y_tr, weight=sw)
dvalid_ab = xgb.DMatrix(X_val[cols_sin_recontacto], label=y_val)
dtest_ab  = xgb.DMatrix(X_test_ext[cols_sin_recontacto])

booster_ab = xgb.train(params, dtrain_ab, num_boost_round=2000,
                       evals=[(dtrain_ab,'train'),(dvalid_ab,'valid')],
                       early_stopping_rounds=50, verbose_eval=False)

y_proba_ab = booster_ab.predict(dtest_ab, iteration_range=(0, booster_ab.best_iteration+1))
y_pred_ab  = (y_proba_ab >= 0.58).astype(int)  # usa tu umbral final para comparar
print(classification_report(y_test, y_pred_ab))


In [ ]:
# === Explicabilidad con SHAP ===
# Importancia global (bar/beeswarm) y dependence plots para variables top.
# Importancias (ya viste el top10 por 'gain')
imp = booster.get_score(importance_type='gain')
print(sorted(imp.items(), key=lambda x: -x[1])[:10])

# (Opcional) SHAP rápido
# !pip install shap
import shap
explainer = shap.TreeExplainer(booster)
shap_values = explainer.shap_values(xgb.DMatrix(X_test_ext))
shap.summary_plot(shap_values, X_test_ext, show=False)  # en Colab, luego plt.show()


In [ ]:
# ===== Reentrenar XGB nativo SIN Recontacto_sf =====
import numpy as np, xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, f1_score, recall_score

# 1) Subset de features SIN Recontacto_sf
cols_sin_recontacto = [c for c in X_train_ext.columns if c != 'Recontacto_sf']
X_tr_full = X_train_ext[cols_sin_recontacto]
X_te_full = X_test_ext[cols_sin_recontacto]

# 2) Split interno p/ early stopping
X_tr, X_val, y_tr, y_val = train_test_split(
    X_tr_full, y_train, test_size=0.10, random_state=42, stratify=y_train
)

# 3) Pesos favoreciendo clase 0
ratio = (y_tr==1).sum() / max(1,(y_tr==0).sum())
sw = np.where(y_tr==0, ratio, 1.0)

# 4) DMatrix
dtrain = xgb.DMatrix(X_tr, label=y_tr, weight=sw)
dvalid = xgb.DMatrix(X_val, label=y_val)
dtest  = xgb.DMatrix(X_te_full)

# 5) Parámetros (mismos que venías usando)
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 5,
    'eta': 0.05,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'min_child_weight': 2,
    'lambda': 1.0,
    'alpha': 0.5,
    'tree_method': 'hist',
    'seed': 42
}

booster_sin = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=2000,
    evals=[(dtrain,'train'),(dvalid,'valid')],
    early_stopping_rounds=50,
    verbose_eval=False
)
print(f"Mejor iteración (sin Recontacto): {booster_sin.best_iteration} | logloss(valid): {booster_sin.best_score:.4f}")

# 6) Predicciones test
y_proba_sin = booster_sin.predict(dtest, iteration_range=(0, booster_sin.best_iteration+1))

# Report con umbral 0.50 (referencia)
y_pred050 = (y_proba_sin >= 0.50).astype(int)
print("📌 Report (0.50) sin Recontacto:")
print(classification_report(y_test, y_pred050))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_sin))

# 7) Elegir umbral para recall_0 ≥ 0.70 (o tu objetivo)
def umbral_por_objetivo(y_true, proba_pos, min_recall0=0.70):
    best = {'thr':0.5, 'rec0':0, 'f1_0':-1}
    for thr in np.linspace(0.1, 0.9, 81):
        y_hat = (proba_pos >= thr).astype(int)
        r0 = recall_score(y_true, y_hat, pos_label=0)
        f10 = f1_score(y_true, y_hat, pos_label=0)
        if r0 >= min_recall0 and f10 > best['f1_0']:
            best = {'thr':thr, 'rec0':r0, 'f1_0':f10}
    return best

best_obj_sin = umbral_por_objetivo(y_test, y_proba_sin, min_recall0=0.70)
print("🎯 Umbral (sin Recontacto) con recall0 ≥ 0.70:", best_obj_sin)

y_hat_obj_sin = (y_proba_sin >= best_obj_sin['thr']).astype(int)
print("📌 Report (umbral objetivo) sin Recontacto:")
print(classification_report(y_test, y_hat_obj_sin))


In [ ]:
# === Bloque misceláneo ===
bins = [-0.1,6,12,24,60,1e9]; labels = ['<6m','6-12m','12-24m','2-5y','5y+']
df['antiguedad_bucket'] = pd.cut(df['antiguedad_cliente_meses'], bins=bins, labels=labels)
# añadila a whitelist, recodificá, rearmá X_train/X_test y vuelve a crear *_freq/_mean0


In [ ]:
# === Limpieza / Normalización: manejo de nulos, tipos, merges y depuración ===
# Objetivo: dejar los datos consistentes antes del EDA/modelado.
# Guardar modelo, umbral y listado de features
booster_sin.save_model("xgb_csat_sin_recontacto.json")

umbral_final = 0.56   # o 0.58, el que elijas
np.save("umbral_final.npy", np.array([umbral_final]))

cols_sin_recontacto = [c for c in X_train_ext.columns if c != 'Recontacto_sf']
with open("features_usadas.txt", "w") as f:
    f.write("\n".join(cols_sin_recontacto))



In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
# ----------------------------
# 1) Asegurar columna de MES
# ----------------------------
# Usaremos 'mes_encuesta' si ya existe. Si no, la creamos desde alguna fecha disponible.
if 'mes_encuesta' not in df.columns:
    # Intentar a partir de fecha_encuesta o columnas similares
    posibles = ['Fecha Registrada corta','Fecha registrada']
    col_fecha = next((c for c in posibles if c in df.columns), None)
    if col_fecha is None:
        raise ValueError("No encuentro una columna de fecha para inferir el mes.")
    df[col_fecha] = pd.to_datetime(df[col_fecha], errors='coerce')
    df['mes_encuesta'] = df[col_fecha].dt.month

# ----------------------------
# 2) Reconstruir df_modelo limpio (whitelist)
# ----------------------------
whitelist = [
    'antiguedad_cliente_meses','rango_etario','genero','ya_es_cliente','ciudad',
    'Producto_sf','Subproduct_sf','grupo_producto','motivo_contacto_categoria',
    'Canal','canal_ticket','Duración (en segundos)','mes_encuesta','dia_semana_encuesta','hora_encuesta'
]
cols_exist = [c for c in whitelist if c in df.columns]
df_modelo = df[cols_exist + ['CSAT_bin']].copy()

# Anti-fuga (por si se coló algo de CSAT o Resolución textual)
ban_patterns = ['csat','satisf','resoluc']
bad = [c for c in df_modelo.columns if any(p in c.lower() for p in ban_patterns) and c!='CSAT_bin']
if bad:
    df_modelo = df_modelo.drop(columns=bad)

# ----------------------------
# 3) Codificar categóricas SOLO en object/category
# ----------------------------
X_raw = df_modelo.drop(columns=['CSAT_bin']).copy()
y_all = df_modelo['CSAT_bin'].astype(int)
for col in X_raw.select_dtypes(include=['object','category']).columns:
    le = LabelEncoder()
    X_raw[col] = le.fit_transform(X_raw[col].astype(str))

# ----------------------------
# 4) Imputación (fit en TRAIN temporal)
# ----------------------------
# Definir máscaras temporales
mask_train = df['mes_encuesta'].isin([2,3,4])  # Feb-Abr
mask_test  = df['mes_encuesta'].isin([5,6])    # May-Jun

imp = SimpleImputer(strategy='mean')
X_train_ts = pd.DataFrame(imp.fit_transform(X_raw[mask_train]), columns=X_raw.columns, index=X_raw[mask_train].index)
X_test_ts  = pd.DataFrame(imp.transform(X_raw[mask_test]),  columns=X_raw.columns, index=X_raw[mask_test].index)
y_train_ts = y_all[mask_train]
y_test_ts  = y_all[mask_test]

print("Temporal shapes:", X_train_ts.shape, X_test_ts.shape)

# ----------------------------
# 5) Encodings adicionales: *_freq y *_mean0 (fit SOLO en train temporal)
# ----------------------------
cand_cols = [c for c in [
    'Producto_sf','Subproduct_sf','grupo_producto','motivo_contacto_categoria',
    'rango_etario','genero','ya_es_cliente','ciudad','Canal','canal_ticket'
] if c in X_train_ts.columns]

def add_freq_and_target_mean(Xtr, Xte, ytr, cols, positive_class=0):
    Xtr = Xtr.copy(); Xte = Xte.copy()
    base_p0 = (ytr == positive_class).mean()
    for col in cols:
        # frequency
        freq = Xtr[col].value_counts(normalize=True)
        Xtr[f'{col}_freq'] = Xtr[col].map(freq)
        Xte[f'{col}_freq'] = Xte[col].map(freq).fillna(0)
        # target-mean (prob de clase 0)
        tmp = pd.DataFrame({col: Xtr[col], 'y': ytr.values})
        mean1 = tmp.groupby(col)['y'].mean()
        prob0 = 1.0 - mean1
        Xtr[f'{col}_mean0'] = Xtr[col].map(prob0)
        Xte[f'{col}_mean0'] = Xte[col].map(prob0).fillna(base_p0)
    return Xtr, Xte

X_tr_ext, X_te_ext = add_freq_and_target_mean(X_train_ts, X_test_ts, y_train_ts, cand_cols, positive_class=0)

# ----------------------------
# 6) Entrenar XGB nativo con early stopping (solo TRAIN temporal)
# ----------------------------
import xgboost as xgb
ratio = (y_train_ts==1).sum() / max(1,(y_train_ts==0).sum())
sw = np.where(y_train_ts==0, ratio, 1.0)

# split interno para early stopping dentro del TRAIN temporal
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_tr_ext, y_train_ts, test_size=0.10, random_state=42, stratify=y_train_ts)

dtrain = xgb.DMatrix(X_tr,  label=y_tr,  weight=np.where(y_tr==0, ratio, 1.0))
dvalid = xgb.DMatrix(X_val, label=y_val)
dtest  = xgb.DMatrix(X_te_ext)

params = {
    'objective':'binary:logistic','eval_metric':'logloss',
    'max_depth':5,'eta':0.05,'subsample':0.9,'colsample_bytree':0.9,
    'min_child_weight':2,'lambda':1.0,'alpha':0.5,'tree_method':'hist','seed':42
}
booster_ts = xgb.train(params, dtrain, num_boost_round=2000,
                       evals=[(dtrain,'train'),(dvalid,'valid')],
                       early_stopping_rounds=50, verbose_eval=False)

print(f"Mejor iteración (temporal): {booster_ts.best_iteration} | logloss(valid): {booster_ts.best_score:.4f}")

# ----------------------------
# 7) Evaluación en TEST temporal (May-Jun)
# ----------------------------
y_proba_ts = booster_ts.predict(dtest, iteration_range=(0, booster_ts.best_iteration+1))
y_pred050  = (y_proba_ts >= 0.50).astype(int)
print("📌 Report temporal (umbral 0.50):")
print(classification_report(y_test_ts, y_pred050))
print("ROC-AUC temporal:", roc_auc_score(y_test_ts, y_proba_ts))

# Umbral por objetivo (ej: recall0 ≥ 0.70)
def umbral_por_objetivo(y_true, proba_pos, min_recall0=0.70):
    best = {'thr':0.5, 'rec0':0, 'f1_0':-1}
    for thr in np.linspace(0.1,0.9,81):
        y_hat = (proba_pos >= thr).astype(int)
        r0 = recall_score(y_true, y_hat, pos_label=0)
        f10 = f1_score(y_true, y_hat, pos_label=0)
        if r0 >= min_recall0 and f10 > best['f1_0']:
            best = {'thr':thr, 'rec0':r0, 'f1_0':f10}
    return best

best_obj_ts = umbral_por_objetivo(y_test_ts, y_proba_ts, min_recall0=0.70)
print("🎯 Umbral temporal con recall0 ≥ 0.70:", best_obj_ts)

y_hat_ts = (y_proba_ts >= best_obj_ts['thr']).astype(int)
print("📌 Report temporal (umbral objetivo):")
print(classification_report(y_test_ts, y_hat_ts))

sns.heatmap(confusion_matrix(y_test_ts, y_hat_ts), annot=True, fmt='d', cmap='Oranges',
            xticklabels=['No Sat','Sat'], yticklabels=['No Sat','Sat'])
plt.title("Matriz — Validación temporal (umbral objetivo)")
plt.xlabel("Predicción"); plt.ylabel("Real"); plt.show()


In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
# Modelo (entrenado en Feb–Abr) + umbral encontrado para May–Jun
booster_ts.save_model("xgb_csat_temporal_febabr.json")

import numpy as np
np.save("umbral_temporal.npy", np.array([float(best_obj_ts['thr'])]))  # ej: 0.66

# Columnas usadas y su orden
with open("features_temporal.txt","w") as f:
    f.write("\n".join(X_tr_ext.columns.astype(str)))


In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
from sklearn.metrics import confusion_matrix

def confusion_clase0(y_true, y_hat):
    # tratamos "0" como la clase positiva
    y_true0 = (y_true == 0).astype(int)
    y_hat0  = (y_hat  == 0).astype(int)
    tn0, fp0, fn0, tp0 = confusion_matrix(y_true0, y_hat0).ravel()
    return tn0, fp0, fn0, tp0  # tp0 = aciertos "No Sat"; fn0 = "No Sat" perdidos

def umbral_por_coste_clase0(y_true, proba_pos, cost_fn=5.0, cost_fp=1.0):
    mejores = {'thr':0.5, 'costo':float('inf'), 'tn0':0,'fp0':0,'fn0':0,'tp0':0}
    import numpy as np
    for thr in np.linspace(0.1, 0.9, 81):
        y_hat = (proba_pos >= thr).astype(int)  # 1="Sat", 0="No Sat"
        tn0, fp0, fn0, tp0 = confusion_clase0(y_true, y_hat)
        costo = cost_fn*fn0 + cost_fp*fp0
        if costo < mejores['costo']:
            mejores = {'thr':float(thr), 'costo':float(costo),
                       'tn0':int(tn0),'fp0':int(fp0),'fn0':int(fn0),'tp0':int(tp0)}
    return mejores

best_cost0 = umbral_por_coste_clase0(y_test_ts, y_proba_ts, cost_fn=5.0, cost_fp=1.0)
best_cost0


In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
from sklearn.metrics import fbeta_score, precision_score, recall_score
import numpy as np

def umbral_por_Fbeta_clase0(y_true, proba_pos, beta=2.0):
    mejor = {'thr':0.5,'fbeta_0':-1,'rec0':0,'prec0':0}
    for thr in np.linspace(0.1, 0.9, 81):
        y_hat = (proba_pos >= thr).astype(int)
        # calculamos F_beta tratando "0" como la clase de interés
        y_true0 = (y_true == 0).astype(int)
        y_hat0  = (y_hat  == 0).astype(int)
        f0 = fbeta_score(y_true0, y_hat0, beta=beta)
        r0 = recall_score(y_true0, y_hat0)
        p0 = precision_score(y_true0, y_hat0, zero_division=0)
        if f0 > mejor['fbeta_0']:
            mejor = {'thr':float(thr),'fbeta_0':float(f0),'rec0':float(r0),'prec0':float(p0)}
    return mejor

best_f2_ts = umbral_por_Fbeta_clase0(y_test_ts, y_proba_ts, beta=2.0)
best_f2_ts


In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
import pandas as pd

def comparar_proporciones(df, col, mask_train, mask_test, top_n=10):
    p_train = df.loc[mask_train, col].value_counts(normalize=True)
    p_test  = df.loc[mask_test,  col].value_counts(normalize=True)
    comp = pd.concat([p_train.rename('train'), p_test.rename('test')], axis=1).fillna(0)
    comp['delta_pp'] = (comp['test'] - comp['train'])*100
    return comp.reindex(comp['delta_pp'].abs().sort_values(ascending=False).index).head(top_n)

# Ejemplos (usa columnas que tengas)
cols_cat = [c for c in ['motivo_contacto_categoria','Producto_sf','Subproduct_sf','Canal','ciudad'] if c in df.columns]

for c in cols_cat:
    print(f"\n>>> Drift top en {c}")
    display(comparar_proporciones(df, c, mask_train, mask_test, top_n=10))


In [ ]:
# === Limpieza / Normalización: manejo de nulos, tipos, merges y depuración ===
# Objetivo: dejar los datos consistentes antes del EDA/modelado.

def confusion_clase0(y_true, y_hat):
    y_true0 = (y_true == 0).astype(int)
    y_hat0  = (y_hat  == 0).astype(int)
    tn0, fp0, fn0, tp0 = confusion_matrix(y_true0, y_hat0).ravel()
    return tn0, fp0, fn0, tp0

def resumen_umbral(y_true, proba_pos, thr, cost_fn=5.0, cost_fp=1.0):
    y_hat = (proba_pos >= thr).astype(int)    # 1=Sat, 0=No Sat
    tn0, fp0, fn0, tp0 = confusion_clase0(y_true, y_hat)
    prec0 = tp0 / (tp0 + fp0 + 1e-9)
    rec0  = tp0 / (tp0 + fn0 + 1e-9)
    f1_0  = 2*prec0*rec0 / (prec0 + rec0 + 1e-9)
    costo = cost_fn*fn0 + cost_fp*fp0
    return {
        'thr': thr, 'prec0': prec0, 'rec0': rec0, 'f1_0': f1_0,
        'fp0': int(fp0), 'fn0': int(fn0), 'tp0': int(tp0), 'tn0': int(tn0), 'costo': float(costo)
    }

umbrales = [0.50, 0.66, 0.73, 0.88]
tabla = [resumen_umbral(y_test_ts, y_proba_ts, t, cost_fn=5.0, cost_fp=1.0) for t in umbrales]
for row in tabla:
    print(row)


In [ ]:
# === Bloque misceláneo ===
# --- Importancias (gain) top 15 ---
imp = booster_ts.get_score(importance_type='gain')
top = sorted(imp.items(), key=lambda x: -x[1])[:15]
for k,v in top:
    print(f"{k}: {v:.4f}")


In [ ]:
# === Explicabilidad con SHAP ===
# Importancia global (bar/beeswarm) y dependence plots para variables top.
# --- SHAP summary plot ---
# !pip install shap

# Usamos el mismo booster_ts y X_te_ext de la validación temporal
explainer = shap.TreeExplainer(booster_ts)
# OJO: SHAP espera matriz/array, pasamos el mismo orden de columnas
X_plot = X_te_ext.copy()
shap_values = explainer.shap_values(xgb.DMatrix(X_plot))

shap.summary_plot(shap_values, X_plot, show=False)
plt.title("SHAP Summary — Test temporal (May–Jun)")
plt.show()


In [ ]:
# === Limpieza / Normalización: manejo de nulos, tipos, merges y depuración ===
# Objetivo: dejar los datos consistentes antes del EDA/modelado.
def confusion_clase0(y_true, y_hat):
    y_true0=(y_true==0).astype(int); y_hat0=(y_hat==0).astype(int)
    tn0, fp0, fn0, tp0 = confusion_matrix(y_true0, y_hat0).ravel()
    return tn0, fp0, fn0, tp0

def fila(y_true, proba_pos, thr, cost_fn=5.0, cost_fp=1.0):
    y_hat = (proba_pos >= thr).astype(int)
    tn0, fp0, fn0, tp0 = confusion_clase0(y_true, y_hat)
    prec0 = tp0/(tp0+fp0+1e-9); rec0 = tp0/(tp0+fn0+1e-9)
    f1_0 = 2*prec0*rec0/(prec0+rec0+1e-9)
    costo = cost_fn*fn0 + cost_fp*fp0
    return [thr, prec0, rec0, f1_0, fp0, fn0, costo]

df_tab = pd.DataFrame(
    [fila(y_test_ts, y_proba_ts, t) for t in [0.50, 0.66, 0.73, 0.88]],
    columns=["Umbral","Prec_0","Recall_0","F1_0","FP_0","FN_0","Costo(5*FN0+FP0)"]
).round(3)
df_tab


In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
from sklearn.metrics import roc_curve, auc, precision_recall_curve
#import matplotlib.pyplot as plt, numpy as np

y0 = (y_test_ts==0).astype(int)          # clase positiva = No Sat
score0 = 1 - y_proba_ts                  # prob(No Sat)

# ROC
fpr, tpr, _ = roc_curve(y0, score0)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr); plt.plot([0,1],[0,1],'--')
plt.title(f"ROC (No Sat) - AUC={roc_auc:.3f}"); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.show()

# Precision-Recall
prec, rec, thr = precision_recall_curve(y0, score0)
plt.plot(rec, prec); plt.title("Precision-Recall (No Sat)")
plt.xlabel("Recall₀"); plt.ylabel("Precision₀")
# marca tus umbrales (pSat→pNoSat = 1-umbral)
for t in [0.66, 0.73]:
    t0 = 1 - t
    j = np.argmin(np.abs(thr - t0))
    plt.scatter(rec[j], prec[j], label=f"thr={t}", s=60)
plt.legend(); plt.show()


In [ ]:
# === Importaciones: librerías usadas en todo el notebook ===
#import seaborn as sns, pandas as pd
dfp = pd.DataFrame({"pSat": y_proba_ts, "y": y_test_ts.map({0:"No Sat",1:"Sat"})})
sns.kdeplot(data=dfp, x="pSat", hue="y", common_norm=False);
for t in [0.66,0.73]: plt.axvline(t, ls='--', alpha=.4)
plt.title("Distribución p(Sat) por clase"); plt.show()


In [ ]:
# === Explicabilidad con SHAP ===
# Importancia global (bar/beeswarm) y dependence plots para variables top.
# summary tipo barra (importancia media |SHAP|)
shap.summary_plot(shap_values, X_te_ext, plot_type="bar", show=False)
plt.title("SHAP Bar — Test temporal"); plt.show()

# dependence plots (relación de 1 variable con el output)
for feat in ["Subproduct_sf_mean0","Duración (en segundos)","Producto_sf_mean0"]:
    shap.dependence_plot(feat, shap_values, X_te_ext, show=False)
    plt.title(f"SHAP Dependence — {feat}"); plt.show()


In [ ]:
# === Limpieza / Normalización: manejo de nulos, tipos, merges y depuración ===
# Objetivo: dejar los datos consistentes antes del EDA/modelado.
y0 = (y_test_ts==0).astype(int)
score0 = 1 - y_proba_ts
order = np.argsort(-score0)              # mayor riesgo primero
y0_sorted = y0.iloc[order].to_numpy()

cum_pos = np.cumsum(y0_sorted)
total_pos = y0.sum()
pct_pobl = np.arange(1, len(y0_sorted)+1)/len(y0_sorted)
rec_cum  = cum_pos/total_pos

plt.plot(pct_pobl, rec_cum, label="Modelo")
plt.plot(pct_pobl, pct_pobl, '--', label="Aleatorio")
plt.xlabel("% población ordenada por riesgo"); plt.ylabel("Recall₀ acumulado")
plt.title("Cumulative Gains — No Sat"); plt.legend(); plt.show()


In [54]:
# === Montaje de Google Drive para acceder/guardar archivos ===
from google.colab import drive
drive.mount('/content/drive')

#!ls "/content/drive/MyDrive/TPF"

!ls -l "/content/drive/MyDrive/TPF/TFM_CSAT_Desde_Drive.ipynb"

#!jupyter nbconvert --to html "/content/drive/MyDrive/TFM_CSAT_Desde_Drive.ipynb" --output "/content/drive/MyDrive/TFM_CSAT.html"

!jupyter nbconvert --to html "/content/drive/MyDrive/TPF/TFM_CSAT_Desde_Drive.ipynb" --output "/content/drive/MyDrive/TPF/TFM_CSAT2.html"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
-rw------- 1 root root 1575653 Sep 18 05:10 /content/drive/MyDrive/TPF/TFM_CSAT_Desde_Drive.ipynb
[NbConvertApp] Converting notebook /content/drive/MyDrive/TPF/TFM_CSAT_Desde_Drive.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 22 image(s).
[NbConvertApp] Writing 2006813 bytes to /content/drive/MyDrive/TPF/TFM_CSAT2.html
